In [1]:
import numpy as np

from live_strat import RSIRisingFalling
from datetime import datetime

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=3000,
)

In [3]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(30, 61, 5),
)

In [4]:
backtest_settings = BacktestSettings(gains_pct_filter=0)

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([1, 0.25]),
    sl_based_on_lookback=np.array([30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1, 2, 3]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [5]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 7
Total order settings to test: 24
Total combinations of settings to test: 168
Total candles: 3,000
Total candles to test: 504,000


In [6]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
49,2,1,21.0,13,8,126.043,61.905,0.768,121.431,2260.426,1260.426
3,0,3,6.0,4,2,16.089,66.667,0.749,9.994,1160.890,160.890
48,2,0,25.0,16,9,131.922,64.000,0.748,115.499,2319.225,1319.225
9,0,9,6.0,5,1,23.036,83.333,0.747,17.977,1230.358,230.358
21,0,21,6.0,4,2,35.423,66.667,0.738,19.836,1354.226,354.226
52,2,4,14.0,8,6,223.502,57.143,0.737,115.852,3235.020,2235.020
53,2,5,14.0,8,6,223.502,57.143,0.737,115.852,3235.020,2235.020
41,1,17,6.0,3,3,51.952,50.000,0.717,25.877,1519.519,519.519
96,4,0,33.0,20,13,176.616,60.606,0.707,342.058,2766.165,1766.165
60,2,12,25.0,16,9,93.176,64.000,0.706,122.450,1931.758,931.758


In [7]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    ind_set_index=2,
    dos_index=1,
    plot_results=True,
    logger_bool=False,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.01,
    sl_based_on_lookback = 30,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.02,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 40.0,
    rsi_length = 14,
)
